# Readme.txt
_Extract from the readme.txt file_

### Structure
The VIPER dataset is split into training, validation, and test set. 
Each subset is further divided into a number of video sequences. Each video
sequence was recorded in 1 of 5 environmental conditions (day, sunset, rain, 
night, snow). The frame rate of each sequence is approximately 15 fps.

### Input images (img)
Each image was recorded in 1080p (1920x1080 pixels). Due to the large number of 
frames, we provide the images in lossy JPG and lossless PNG format. 

### 2D/3D Bounding boxes (bb)
We provide bounding boxes in 2D (on the image plane) and in 3D 
(in camera coordinate frame) for a subset of the semantic classes. The 
annotations are stored as one CSV file per frame. Each row in one of the files
corresponds to a single object instance. The format for the columns is
classID, instanceID, 2D bounding box (4 values), 3D bounding box in model 
coordinate frame (6 values), matrix to transform 3D bounding box into camera 
coordinate frame (16 values).
An example for visualizing the bounding boxes for one frame is given in 
'drawBoundingBox.m'.



## Import Weather Conditions
weather.txt : https://drive.google.com/uc?id=1fJTx6-bfKYQs1CQD1yh7RsTIjhR3n0y0&export=download

In [4]:
# Import the Weather information from weather.txt
path_weather_file = '/Users/lucas/Documents/Master Thesis/gta_dataset/train/weather.txt'

# List of the different weather conditions available
list_weather = ['day', 'sunset', 'night', 'rain', 'snow']

# Initialized dict to store weather information per folder
weather_dict = {w: [] for w in list_weather}

# File Organization [id_set, weather]:
with open(path_weather_file) as f:
    for line in f:
        # Parse line
        line_parsed = line.strip().split(',')

        # Fill the dictionary with the values
        weather_dict[line_parsed[1]].append(line_parsed[0])

# Print repartition of the weather conditions
print('Number of sequences per weather condition:')
for w, l in weather_dict.items():
    print(' \'-> ' + w + '(' + str(len(l)) + '): ' + str(l))
          

Number of sequences per weather condition:
 '-> rain(8): ['30', '31', '59', '60', '61', '62', '63', '64']
 '-> day(19): ['01', '02', '03', '04', '05', '06', '44', '45', '46', '47', '48', '49', '50', '51', '65', '66', '67', '68', '69']
 '-> sunset(17): ['07', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29']
 '-> night(21): ['08', '09', '10', '11', '12', '13', '52', '53', '54', '55', '56', '57', '58', '70', '71', '72', '73', '74', '75', '76', '77']
 '-> snow(12): ['32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43']


# Import Labels
train_bb.zip: https://drive.google.com/uc?id=1r9xZmfAT75geV2ieotND4U7OMbBkp2OQ&export=download

In [13]:
# Correspondance between KITTI classes and GTA classses
classes_corres = {
    'Car': ['car'],
    'Van': ['van'],
    'Truck': ['truck'],
    'Pedestrian': ['person'],
    'Person_sitting': [],
    'Cyclist': ['bicycle'],
    'Tram': ['train'],
    'Misc': ['motorcycle', 'bus', 'trailer', 'animal'],
    'DontCare': ['ambiguous'],
    'Ignore': ['unlabeled', 'sky', 'road', 'sidewalk', 'railtrack',\
               'terrain', 'tree', 'vegetation', 'building',\
               'infrastructure', 'fence', 'billboard', 'trafficlight',\
               'trafficsign', 'mobilebarrier', 'firehydrant', 'chair',\
               'trash', 'trashcan', 'plane', 'boat']
}

# Inverse classes_corres
gta2kitti = {}
for k, v_list in classes_corres.items():
    if not k == 'Ignore': # remove 'Ignore' classes
        for v in v_list:
            gta2kitti[v] = k
print(gta2kitti)

{'motorcycle': 'Misc', 'person': 'Pedestrian', 'bicycle': 'Cyclist', 'trailer': 'Misc', 'bus': 'Misc', 'ambiguous': 'DontCare', 'animal': 'Misc', 'train': 'Tram', 'car': 'Car', 'truck': 'Truck', 'van': 'Van'}


In [15]:
# Import the Classes information from classses.csv
path_classes_file = '/Users/lucas/Documents/Master Thesis/gta_dataset/readme/classes.csv'

# Extract correspondance between id and classname in the classes.csv
id2class = {}
with open(path_classes_file) as f:
    for line in f:
         # Parse line
        line_parsed = line.strip().split(',')
        
        if not line_parsed[0] == 'id': # Ignore header
            id2class[line_parsed[0]]= line_parsed[1]
 
# Create dictionnary to extract future classes from GTA dataset
classes_dict = {k:{'kitti': gta2kitti[v], 'gta': v} for k, v in id2class.items() if not v in classes_corres['Ignore']}        
print(classes_dict)
        



{'29': {'gta': 'train', 'kitti': 'Tram'}, '21': {'gta': 'animal', 'kitti': 'Misc'}, '22': {'gta': 'bicycle', 'kitti': 'Cyclist'}, '28': {'gta': 'trailer', 'kitti': 'Misc'}, '24': {'gta': 'car', 'kitti': 'Car'}, '26': {'gta': 'bus', 'kitti': 'Misc'}, '25': {'gta': 'van', 'kitti': 'Van'}, '27': {'gta': 'truck', 'kitti': 'Truck'}, '23': {'gta': 'motorcycle', 'kitti': 'Misc'}, '20': {'gta': 'person', 'kitti': 'Pedestrian'}, '1': {'gta': 'ambiguous', 'kitti': 'DontCare'}}


From KITTI:
 ↳ At least one object per image (Car, Cyclist or Pedestrian in  
 Easy: Min. bounding box height: 40 Px, Max. occlusion level: Fully visible, Max. truncation: 15 %
 Moderate: Min. bounding box height: 25 Px, Max. occlusion level: Partly occluded, Max. truncation: 30 %
 Hard: Min. bounding box height: 25 Px, Max. occlusion level: Difficult to see, Max. truncation: 50 %
 
 Correspondance KITTI classes with GTA_Dataset:
 'Car', 'Van', 'Truck', 'Pedestrian', 'Person_sitting', 'Cyclist', 'Tram', 'Misc' or 'DontCare'
 In KITII buses and motorcycles and trailer are considered as MISC (see 00098.png) --> try to avoid them in the selection of the images
 
 Check if train in GTA can be seen as tram in KITTI or discard.
 
 DontCare are ambiguous or too small labels
 

In [ ]:
# Function to import the useful bboxes
# From KITTI :
#  ↳

In [ ]:
# Crop image and reposition the labels.
# convert ../Images/000[-9][0-9].png -resize 1242x683^ -crop 1242x375+0+84 -set filename:f '%t_1242x375.%e' +adjoin '%[filename:f]'